In [1]:
%%capture
!!pip install -q git+https://github.com/keras-team/keras-nlp.git --upgrade

In [2]:
import pandas as pd
import numpy as np
import os
import pickle

import keras_nlp
import tensorflow as tf
from tensorflow.keras import regularizers
from keras.regularizers import l2, l1

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

Using TensorFlow backend


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [152]:
EXERCISE = 'Es1' # Use Es1, Es2, Es3, Es4, Es5, or All to train a model on all exercises

# Mapping of exercise names to their maximum lengths
max_length_mapping = {
        "Es1": 1515,
        "Es2": 1668,
        "Es3": 1518,
        "Es4": 1988,
        "Es5": 1022
    }

# Mapping of exercise names to their temporal window configurations
temporal_windows_mapping = { #[num_windows, window_size]
        "Es1": [5,303],
        "Es2": [3,556],
        "Es3": [11,138],
        "Es4": [4,497],
        "Es5": [7,146],
    }

def get_max_length(exercise):
  if exercise == "All":
    return max(max_length_mapping.values())
  else:
    return max_length_mapping[exercise]

def get_temporal_windows(exercise):
  if exercise == "All":
    max_es = max(max_length_mapping.keys(), key=max_length_mapping.get)
    return temporal_windows_mapping[max_es]
  else:
    return temporal_windows_mapping[exercise]

max_exercise_length = get_max_length(EXERCISE)
temporal_windows = get_temporal_windows(EXERCISE)

print(f"Max frames: {max_exercise_length}")
print(f"Number of temporal windows: {temporal_windows[0]} - temporal window size: {temporal_windows[1]}")

Max frames: 1515
Number of temporal windows: 5 - temporal window size: 303


# Load and Prepare Data

In [153]:
df = pd.read_csv("/content/drive/MyDrive/rehab-ai-data/KiMoRe_final/KiMoRe_data_movenet_features.csv")
if EXERCISE != "All":
  df = df[df['exercise']==EXERCISE]

In [154]:
df.head()

,ID,exercise,video,joint_positions,joint_features,clinical_score,#frames
2,P_ID11,Es1,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,14.666667,529
8,P_ID16,Es1,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,37.000000,702
14,P_ID10,Es1,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,34.308180,606
18,P_ID4,Es1,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,14.000000,842
24,P_ID3,Es1,NaN,NaN,NaN,-1.000000,0


In [155]:
def get_dataframe_cols():
  KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
  }
  df_cols = []
  for keypoint_name in KEYPOINT_DICT:
    df_cols.append(f"{keypoint_name}_y")
    df_cols.append(f"{keypoint_name}_x")
    df_cols.append(f"{keypoint_name}_confidence")
  return df_cols

In [156]:
all_cols = get_dataframe_cols()
face_cols = all_cols[:15]
cols_drop = face_cols
print(f"Dropping {len(cols_drop)} columns.")

Dropping 15 columns.


In [180]:
print(f"Using exercise: {EXERCISE}")
print(f"Maximum video length: {max_exercise_length}")

# Settings for training configuration
smoothing_window=10
use_joint_positions=True
use_joint_features=False
smooth_joint_positions=False
smooth_joint_features=False

print(f"Smoothing window is set to {smoothing_window}.")
print(f"Joint positions data {'is' if use_joint_positions else 'is not'} being used.")
if use_joint_positions:
  print(f"Joint positions data is being used {'without' if not smooth_joint_positions else 'with'} smoothing.")

print(f"Joint features data {'is' if use_joint_features else 'is not'} being used.")
if use_joint_features:
  print(f"Joint features data is being used {'without' if not smooth_joint_features else 'with'} smoothing.")

def prepare_data(df, exercise_video_max_len):
  """Prepares data for training or evaluation.

  Args:
    df: DataFrame containing exercise data.
    exercise_video_max_len: Maximum length of exercise videos.

  Returns:
    A tuple of (data, padding_masks), labels:
      data: A NumPy array of shape (num_samples, max_length, num_features) containing prepared data.
      padding_masks: A NumPy array of shape (num_samples, max_length) indicating padded elements.
      labels: A NumPy array of shape (num_samples) containing clinical scores.
  """

  # Ensure at least one data type is used
  if not use_joint_positions and not use_joint_features:
    print("At least one of use_joint_positions and use_joint_features should be True!")
    return None, None

  data = []
  labels = []
  padding_masks = []
  for index, row in df.iterrows():
    joint_positions_path = row['joint_positions']
    joint_features_path = row['joint_features']
    if joint_positions_path is np.NAN:
      continue
    clinical_score = row['clinical_score']
    video_length = row['#frames']

    joint_positions_data = None
    # Load joint positions data if needed
    if use_joint_positions:
        joint_positions_data = pd.read_csv(joint_positions_path)
        joint_positions_data = joint_positions_data.drop(cols_drop, axis=1)
        if smooth_joint_positions:
          for col in joint_positions_data.columns:
              joint_positions_data[col] = joint_positions_data[col].rolling(smoothing_window).mean()
        joint_positions_data = joint_positions_data.to_numpy()

    joint_features_data = None
    # Load joint features data if needed
    if use_joint_features:
        joint_features_data = pd.read_csv(joint_features_path)
        if smooth_joint_features:
          for col in joint_features_data.columns:
              joint_features_data[col] = joint_features_data[col].rolling(smoothing_window).mean()
        joint_features_data = joint_features_data.to_numpy()

    data_to_use = None
    # Combine data if both are needed
    if use_joint_positions and use_joint_features:
        data_to_use = np.concatenate((joint_positions_data, joint_features_data), axis=1)
    elif use_joint_positions:
        data_to_use = joint_positions_data
    else:
        data_to_use = joint_features_data

    # Pad data to fixed length and create padding masks
    padding_length = exercise_video_max_len - video_length
    padding_mask = np.zeros((video_length + padding_length))
    padding_mask[-padding_length:] = 1 # Set padding elements to 1

    # Pad data with zeros
    data_to_use_padded = np.pad(data_to_use, ((0, padding_length), (0, 0)), mode='constant', constant_values=0)

    data.append(data_to_use_padded)
    labels.append(clinical_score)
    padding_masks.append(padding_mask)

  data = np.array(data)
  labels = np.array(labels)
  padding_masks = np.array(padding_masks)

  data = np.nan_to_num(data) # Replace NaN values with numerical equivalents
  labels = np.nan_to_num(labels)

  print("Data Shape:", data.shape)
  print("Labels Shape:", labels.shape)
  print("Padding Masks Shape:", padding_masks.shape)

  return (data, padding_masks), labels

Using exercise: Es1
Maximum video length: 1515
Smoothing window is set to 10.
Joint positions data is being used.
Joint positions data is being used without smoothing.
Joint features data is not being used.


In [181]:
(all_data, all_padding), all_labels = prepare_data(df, max_exercise_length)

Data Shape: (72, 1515, 36)
Labels Shape: (72,)
Padding Masks Shape: (72, 1515)


# Build Model

In [182]:
NUM_COLS = all_data[0].shape[1]
NUM_WINDOWS = temporal_windows[0]
WINDOW_SIZE = temporal_windows[1]
NUM_HEADS = 4
D_MODEL = NUM_COLS // 4
if use_joint_features and not use_joint_positions:
  D_MODEL = NUM_COLS // 2 # if using joint features only then NUM_COLS will already be small enough
LEARNING_RATE = 0.001

print(f"Input Data Columns: {NUM_COLS}")
print(f"Number of Temporal Windows: {NUM_WINDOWS}")
print(f"Window Size: {WINDOW_SIZE}")
print(f"Number of Attention Heads: {NUM_HEADS}")
print(f"Intermediate Model Dimension: {D_MODEL}")
print(f"Learning Rate: {LEARNING_RATE}")

Input Data Columns: 36
Number of Temporal Windows: 5
Window Size: 303
Number of Attention Heads: 4
Intermediate Model Dimension: 9
Learning Rate: 0.001


In [183]:
# Define input layers for data and padding masks
inputs = tf.keras.Input(shape=(all_data[0].shape[0], all_data[0].shape[1]), name='orignal_data')
masks = tf.keras.Input(shape=(all_padding.shape[1]), name='padding_masks')

# Split data into windows
windows = tf.split(inputs, NUM_WINDOWS, axis=1)
print("Windows:")
for window in windows:
  print(window.shape)

# Split masks into windows
windows_masks = tf.split(masks, NUM_WINDOWS, axis=1)
print("Windows Masks:")
for mask in windows_masks:
  print(mask.shape)


# Create embedding layers for temporal windows
embedding_layer = tf.keras.layers.Dense(D_MODEL*2, activation='relu')
if not use_joint_positions and use_joint_features:
  embedding_layer = tf.keras.layers.Dense(D_MODEL, activation='relu')
embedding_layer3 = tf.keras.layers.Dense(D_MODEL, activation='relu')

# Apply embedding layers to each window
embeddings = []
for window in windows:
    embedding = embedding_layer(window)
    if use_joint_positions:
      embedding = embedding_layer3(embedding)
    embeddings.append(embedding)

print("Embeddings:")
for embd in embeddings:
  print(embd.shape)

# Create positional embeddings to encode position within windows
positional_embedding_layer = tf.keras.layers.Embedding(input_dim=WINDOW_SIZE, output_dim=D_MODEL)
positional_embeddings = []
for i in range(NUM_WINDOWS):
    positional_embedding = positional_embedding_layer(tf.range(WINDOW_SIZE))
    positional_embeddings.append(positional_embedding)

print("Positional Embeddings:")
for pos_embd in positional_embeddings:
  print(pos_embd.shape)

# Combine temporal and positional embeddings
embeddings_all = [embedding + positional_embedding for embedding, positional_embedding in zip(embeddings, positional_embeddings)]
print("All Embeddings:")
for embd in embeddings_all:
  print(embd.shape)

# Encode windows using a Transformer encoder
transformer_encoder_layer = keras_nlp.layers.TransformerEncoder(intermediate_dim=D_MODEL, num_heads=NUM_HEADS)
encoded = [transformer_encoder_layer(window_embd, window_mask) for window_embd, window_mask in zip(embeddings, windows_masks)]
print("Encodings:")
for enc in encoded:
  print(enc.shape)

# Concatenate encoded windows and flatten
concat_output = tf.concat(encoded, axis=1)
print(f"Concat: {concat_output.shape}")
flatten_output = tf.keras.layers.Flatten()(concat_output)
print(f"Flatten: {flatten_output.shape}")

# Dense layers before final prediction
dense_output = tf.keras.layers.Dense(4970, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(flatten_output)
dense_output = tf.keras.layers.Dense(621, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(dense_output)
dense_output = tf.keras.layers.Dense(77, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(dense_output)
print(f"Final Dense: {dense_output.shape}")
output = tf.keras.layers.Dense(1)(dense_output)

model = tf.keras.Model(inputs=[inputs, masks],
                       outputs=output,
                       name='transformer_model')

Windows:
(None, 303, 36)
(None, 303, 36)
(None, 303, 36)
(None, 303, 36)
(None, 303, 36)
Windows Masks:
(None, 303)
(None, 303)
(None, 303)
(None, 303)
(None, 303)
Embeddings:
(None, 303, 9)
(None, 303, 9)
(None, 303, 9)
(None, 303, 9)
(None, 303, 9)
Positional Embeddings:
(303, 9)
(303, 9)
(303, 9)
(303, 9)
(303, 9)
All Embeddings:
(None, 303, 9)
(None, 303, 9)
(None, 303, 9)
(None, 303, 9)
(None, 303, 9)
Encodings:
(None, 303, 9)
(None, 303, 9)
(None, 303, 9)
(None, 303, 9)
(None, 303, 9)
Concat: (None, 1515, 9)
Flatten: (None, 13635)
Final Dense: (None, 77)


In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True, show_layer_activations=True)

# Plot Functions

In [162]:
def plot_predictions(train_preds, train_labels, test_preds, test_labels, fold):
  # Create a figure with two subplots
  plt.figure(figsize=(8, 8))
  plt.suptitle(f'{EXERCISE} - Fold {fold}',fontsize=20)

  # Plot training set predictions and labels
  plt.subplot(2, 1, 1)
  plt.plot(train_preds, 's', color='red', label='Prediction', linestyle='None', alpha=0.5, markersize=6)
  plt.plot(train_labels, 'o', color='green', label='Clinical Score', alpha=0.4, markersize=6)
  plt.title('Training Set', fontsize=18)
  plt.xlabel('Sequence Number', fontsize=16)
  plt.ylabel('Clinical Score Scale', fontsize=16)
  plt.legend(loc=3, prop={'size': 14})

  # Plot testing set predictions and labels
  plt.subplot(2, 1, 2)
  plt.plot(test_preds, 's', color='red', label='Prediction', linestyle='None', alpha=0.5, markersize=6)
  plt.plot(test_labels, 'o', color='green', label='Clinical Score', alpha=0.4, markersize=6)
  plt.title('Testing Set', fontsize=18)
  plt.xlabel('Sequence Number', fontsize=16)
  plt.ylabel('Clinical Score Scale', fontsize=16)
  plt.legend(loc=3, prop={'size': 14})

  # Adjust layout and save figure
  plt.tight_layout()
  fig_title = f'{EXERCISE}_fold{fold}_pred_plot'
  plt.savefig(f'/content/drive/MyDrive/rehab-ai-data/saved_models_images/temp/{fig_title}.png', dpi=300)
  plt.show()

In [163]:
def plot_history(history, ptype, fold=None):
  # Extract the metric history for training
  type_history = history.history[ptype]

  epochs = range(len(type_history))
  plt.plot(epochs, type_history, label=f'Training {ptype.capitalize()}')

  # If a fold is specified, add validation history
  if fold:
    type_history_val = history.history[f'val_{ptype}']
    plt.plot(epochs, type_history_val, label=f'Validation {ptype.capitalize()}')
    plt.title(f'Training and Validation {ptype.capitalize()}')
    plt.suptitle(f'{EXERCISE} - Fold {fold}')
  else:
    plt.title(f'{EXERCISE} {ptype.capitalize()}')

  # Add labels and legend
  plt.xlabel('Epoch')
  plt.ylabel(f'{ptype.capitalize()}')
  plt.legend()

  # Create and save the plot
  fig_title = f'{EXERCISE}_{ptype}_plot'
  if fold:
    fig_title += f'_fold{fold}'
  plt.savefig(f'/content/drive/MyDrive/rehab-ai-data/saved_models_images/temp/{fig_title}.png', dpi=300)
  plt.show()

# Cross Validation

In [184]:
class PrintEpochs(tf.keras.callbacks.Callback):
    """Prints progress for specific epochs during training."""
    def on_epoch_end(self, epoch, logs=None):
        if epoch in [1, 25, 50, 75, 100]:
              values = ", ".join([f"{key}: {value:.4f}" for key, value in logs.items()])
              print(f"Epoch {epoch}: {values}")

def cross_validate(model, data, labels, padding_masks, k=5):
  """Performs k-fold cross-validation on the model."""
  y_true, y_pred, histories = list(), list(), list()
  i = 1
  kfold = KFold(n_splits=k, random_state=0, shuffle=True)
  print(f"Cross Validating Model Using {k} Folds...")
  for train_idx, val_idx in kfold.split(data):
    print(f"---------------- Fold {i} ----------------")

    # Split data for training and validation
    X_train, X_val = data[train_idx], data[val_idx]
    padding_train, padding_val = padding_masks[train_idx], padding_masks[val_idx]
    y_train, y_val = labels[train_idx], labels[val_idx]

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='mse', metrics=['mae'])

    history = model.fit([X_train, padding_train], y_train, epochs=100, validation_data=([X_val, padding_val], y_val), verbose=0, callbacks=[PrintEpochs()])

    # Make predictions on training and validation sets
    train_pred = model.predict([X_train, padding_train])
    val_pred = model.predict([X_val, padding_val])

    # Store true labels, predictions, and history for later analysis
    y_true.extend(y_val)
    y_pred.extend(val_pred)
    histories.append(history)

    # Calculate and print MAE for the fold
    fold_mae = mean_absolute_error(y_val, val_pred)
    print(f'- MAE of fold {i} = {fold_mae}')

    plot_history(history, 'loss', i)
    plot_history(history, 'mae', i)
    plot_predictions(train_pred, y_train, val_pred, y_val, i)
    i = i+1

  # Calculate overall MAE across all folds
  mae = mean_absolute_error(y_true, y_pred)
  print(f'OOF MAE = {mae}')

In [ ]:
cross_validate(model, all_data, all_labels, all_padding)

# Final Model Training

In [166]:
for i in range(len(model.weights)):
    model.weights[i]._handle_name = str(i) + '__' + model.weights[i].name

In [167]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='mse', metrics=['mae'])

In [168]:
checkpoint_filepath = f'/content/drive/MyDrive/rehab-ai-data/saved_models_weights/ml_model_{EXERCISE}_Weights.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mae',
    save_best_only=False)

In [ ]:
%%time
history = model.fit([all_data, all_padding], all_labels, epochs=100,
                     callbacks=[model_checkpoint_callback])

In [ ]:
model.summary()

# Model History

In [ ]:
plot_history(history, 'loss')

In [ ]:
plot_history(history, 'mae')

# Saving the model

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
!mkdir -p saved_model
model.save(f'/content/drive/MyDrive/rehab-ai-data/saved_models_weights/ml_model_{EXERCISE}.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
